# Interactive basis plaything

Allows you to interactively move the basis points (black dots) like in the animation below.

![morphing animation](morphing.gif)

In [2]:
import numpy as np
import bqplot
from bqplot import pyplot as plt
from IPython.display import display
from ipywidgets import Layout
import numpy as np
import pandas as pd
%matplotlib notebook
import sys

try:
    from madminer.morphing import SimpleMorpher
except ModuleNotFoundError:
    sys.path.append('../..')
    from madminer.morphing import SimpleMorpher
    

## Settings

In [3]:
n_bases = 1
n_resolution = 50

## Preparation

In [4]:
morpher = SimpleMorpher(parameter_max_power=[4,4],
                  parameter_range=[(-1.,1.), (-1.,1.)])
_ = morpher.find_components(max_overall_power=4)

In [5]:
xi = np.linspace(-1.,1.,n_resolution)
yi = np.linspace(-1.,1.,n_resolution)
xx, yy = np.meshgrid(xi, yi)
xx = xx.reshape((-1,1))
yy = yy.reshape((-1,1))
theta_evaluation = np.hstack([xx, yy])

x_updated = False
y_updated = False

## Basis evaluation

In [6]:
def evaluate_basis(basis=None):
    
    # Optimization
    if basis is None:
        basis = morpher.optimize_basis()

    # Evaluate basis
    squared_weights = []

    for theta in theta_evaluation:
        wi = morpher.calculate_morphing_weights(theta, basis)
        squared_weights.append(np.sum(wi*wi)**0.5)

    squared_weights = np.array(squared_weights).reshape((n_resolution,n_resolution))
    
    return basis, squared_weights

## Initial set up

In [7]:
basis, squared_weights = evaluate_basis(None)

## Interactive tool

In [8]:
def update(change):
    global basis, squared_weights, x_updated, y_updated
    
    variable = change['name']
    values = change['new']
    
    if variable == 'x':
        basis[:,0] = values
        x_updated = True
    elif variable == 'y':
        basis[:,1] = values
        y_updated = True
    
    if x_updated and y_updated:
        basis, squared_weights = evaluate_basis(basis)
        heat.color = np.log(squared_weights) / np.log(10)
        
        x_updated = False
        y_updated = False

In [9]:
x_sc = bqplot.scales.LinearScale(min=-1., max=1.)
y_sc = bqplot.scales.LinearScale(min=-1., max=1.)
c_sc = bqplot.scales.ColorScale(min=0, max=1., scheme='YlOrRd')

heat = bqplot.GridHeatMap(color=np.log(squared_weights) / np.log(10),
                          scales={'row': x_sc, 'column': y_sc, 'color': c_sc},
                          row=yi,
                          column=xi,
                          stroke=None)

scatter = bqplot.Scatter(colors=['black']*len(basis[:,0]),
                         x=basis[:,0],
                         y=basis[:,1],
                         scales={'x': x_sc, 'y': y_sc})

ax_x = bqplot.Axis(scale=x_sc, label='fW')
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical', label='fWW')
ax_c = bqplot.ColorAxis(scale=c_sc,
                        orientation='vertical', 
                        side='right')

fig = bqplot.Figure(marks=[heat, scatter],
                    axes=[ax_x, ax_y, ax_c],
                    layout=Layout(width='600px', height='600px'))

display(fig)

scatter.observe(update, ['y','x'])
scatter.enable_move = True


Figure(axes=[Axis(label='fW', scale=LinearScale(max=1.0, min=-1.0)), Axis(label='fWW', orientation='vertical', scale=LinearScale(max=1.0, min=-1.0)), ColorAxis(orientation='vertical', scale=ColorScale(max=1.0, min=0.0, scheme='YlOrRd'), side='right')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='600px', width='600px'), marks=[GridHeatMap(anchor_style={'fill': 'white', 'stroke': 'blue'}, color=array([[0.83117436, 0.78831682, 0.74910515, ..., 1.47139559, 1.49938042,
        1.52994155],
       [0.74093233, 0.68871653, 0.63999085, ..., 1.41648728, 1.44798882,
        1.48254346],
       [0.65348788, 0.59024641, 0.52931563, ..., 1.36100636, 1.39699331,
        1.43644048],
       ...,
       [0.68375215, 0.65271783, 0.62956666, ..., 0.07031715, 0.31271596,
        0.51256153],
       [0.80065495, 0.77371959, 0.75100878, ..., 0.31909862, 0.47849569,
        0.62329231],
       [0.90557096, 0.88114145, 0.85908855, ..., 0.5208593 , 0.62717326,
        0.73296442]]), column=array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ]), interactions={'hover': 'tooltip'}, row=array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ]), scales={'row': LinearScale(max=1.0, min=-1.0), 'column': LinearScale(max=1.0, min=-1.0), 'color': ColorScale(max=1.0, min=0.0, scheme='YlOrRd')}, scales_metadata={'row': {'orientation': 'vertical', 'dimension': 'y'}, 'column': {'orientation': 'horizontal', 'dimension': 'x'}, 'color': {'dimension': 'color'}}, stroke=None, tooltip_style={'opacity': 0.9}), Scatter(colors=['black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=1.0, min=-1.0), 'y': LinearScale(max=1.0, min=-1.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([-0.31814444,  0.77526178, -0.54577662,  0.82610588,  0.95368028,
        0.28903275,  0.62668084, -0.98731433, -0.42474698, -0.71250242,
       -0.9530149 , -0.18384336,  0.19845547, -0.82119701, -0.43596096]), y=array([-0.85120724,  0.20388534, -0.37695552,  0.95105939,  0.8030668 ,
        0.00739771, -0.46558044, -0.02907956, -0.18310758, -0.83692491,
        0.74659957, -0.45950605,  0.84083695,  0.17536081,  0.82660613]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False,